## Accessing SRA Controlled Access data
This notebook explores using GA4GH DRS to access data stored in the cloud for a controlled access dbGaP project - [phs001554](https://www.ncbi.nlm.nih.gov/projects/gap/cgi-bin/study.cgi?study_id=phs001554.v1.p1) "Detection of Colorectal Cancer Susceptibility Loci Using Genome-Wide Sequencing".

A query using the GA4GH Search API on subject and file attributes identifies some DRS ids. 

>Note: The subject and sample data searched through the initial query are on a scrambled copy of the data so that no controlled access records remain intact. Please do not attempt to draw meaningful associations between the subject and sample attributes with the genomic data; this would lead to erroneous conclusions. These examples serve only to illustrate future possibilities, in the hope of identifying ways to make those possibilities real.
>
>The sequence files used in this notebook are also under controlled access. You will not be able to access those files unless you have been granted access through dbGaP to the phs001554 study. If you have that access you will also have access to the non-scrambled subject and sample data. If that is the case, we would be pleased to hear of your interest, and the possibility of collaborating to ensure the GA4GH tools described here enable your studies. Please see [form to register interest](https://docs.google.com/forms/d/e/1FAIpQLSfmmc3VKd6ANdzaVMyelT3c9gIWuoS4ZwT0vsqD-o2ZRxJf7A/viewform)

In [2]:
from fasp.search import DiscoverySearchClient
cl = DiscoverySearchClient('https://ga4gh-search-adapter-presto-public.prod.dnastack.com/')
query = '''select acc, sa.sample_id, sra_drs_id, sex, age
from search_cloud.cshcodeathon.gecco_sra_drs_index i
join dbgap_demo.scr_gecco_susceptibility.sample_multi sa on sa.sample_id = i.sample_id
join dbgap_demo.scr_gecco_susceptibility.subject_phenotypes_multi su on su.dbgap_subject_id = sa.dbgap_subject_id
where age between 50 and 55
and affection_status = 'Case'
and file_type = 'cram' limit 10'''
df = cl.runQuery(query, returnType='dataframe')
df

_Retrieving the query_
____Page1_______________
____Page2_______________
____Page3_______________
____Page4_______________
____Page5_______________
____Page6_______________


,acc,sample_id,sra_drs_id,sex,age
0,SRR7271762,117454,4083cc9baac81657edb627635fcfbeb0,Male,54
1,SRR7271780,117477,eb765677453c85f573d744be63a4530e,Female,55
2,SRR7271789,117486,2b070565053062762fed8a1f61fd5c91,Male,55
3,SRR7271798,117497,9324abd1277f50472208dfb6ac15af64,Female,53
4,SRR7271817,117523,3c6840438950a0cdd0fa072dd35854cf,Female,55
5,SRR7271822,117529,4ea5d13b1ede2a4c1578b27853df23d5,Male,50
6,SRR7271837,117546,9e3c1dc17119d6037b14e8269f41f359,Female,55
7,SRR7271841,117550,b4b831b0d4af57754817ef18bc1d4ab3,Female,55
8,SRR7271848,117559,650c2f4f184b040497e0822fe4460874,Female,51
9,SRR7271868,117581,f806c96f31b51747334681a85128eee2,Male,55


In [2]:
from fasp.loc import SRADRSClient
drsClient = SRADRSClient('https://locate.be-md.ncbi.nlm.nih.gov', public=True)
# extract an example if from the results above
example_id = df.at[3,'sra_drs_id']
# Use DRS to find locations for the file
res_list = drsClient.getObject(example_id)
res_list

{'access_methods': [{'access_id': '9debe860de9a336e9a291566a8349e4e668ce72b',
   'region': 's3.us-east-1',
   'type': 'https'},
  {'access_id': 'd97c4f9ef829579d9e59888deec15d66799af278',
   'region': 'gs.US',
   'type': 'https'}],
 'checksums': [{'checksum': '9324abd1277f50472208dfb6ac15af64',
   'type': 'md5'}],
 'created_time': '2018-06-11T10:26:47Z',
 'id': '9324abd1277f50472208dfb6ac15af64',
 'name': '117497.recal.cram',
 'self_url': 'drs://locate.md-be.ncbi.nlm.nih.gov/9324abd1277f50472208dfb6ac15af64',
 'size': 36933985367}

Use custom client method to get a URL to the file in a specified region.

In [3]:
drsClient.getAccessURLRegion(example_id, 's3.us-east-1')


Unauthorized for that DRS id


In [4]:
drsClient.getAccessURL(example_id,'d97c4f9ef829579d9e59888deec15d66799af278')

Unauthorized for that DRS id


### For interest: Compare the same file on Seven Bridges

In [3]:
query2 = "select * from dbgap_demo.scr_gecco_susceptibility.sb_drs_index where sample_id = '117497' and file_type = 'cram'"
cl.runQuery(query2)

_Retrieving the query_
____Page1_______________
____Page2_______________
____Page3_______________
____Page4_______________
____Page5_______________


[['117497', 'cram', '5ba922a0e4b0db63859cd973']]

In [6]:
from fasp.loc import sbcgcDRSClient
drsClient = sbcgcDRSClient('/Users/forei/.keys/sevenbridges_keys.json', 's3')
drsClient.getObject('5ba922a0e4b0db63859cd973')

{'id': '5ba922a0e4b0db63859cd973',
 'name': '117497.recal.cram',
 'size': 36933985367,
 'checksums': [{'type': 'etag',
   'checksum': '3e4e93345c7b74540706416f51959092-4403'}],
 'self_uri': 'drs://cgc-ga4gh-api.sbgenomics.com/5ba922a0e4b0db63859cd973',
 'created_time': '2018-09-24T17:45:04Z',
 'updated_time': '2018-11-09T15:56:37Z',
 'mime_type': 'application/json',
 'access_methods': [{'type': 's3',
   'region': 'us-east-1',
   'access_id': 'aws-us-east-1'}]}

### For interest: Compare the same file on NCI Data Commons Framework (Gen3)
To do.